In [3]:
import pandas as pd
import numpy as np
import detector
import tree_selection
import json
import os

from sklearn import preprocessing
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
import matplotlib.pyplot as plt


base_path = os.getcwd()

# 读取数据字典

In [4]:
fea_dict_df = pd.read_excel('金融特征库mapping表to元昕更新v5_汇总.xlsx')
fea_dict_df['feature_code'] = fea_dict_df['feature_code'].astype(str)
fea_dict_df_list = fea_dict_df['feature_id'].tolist()
print(len(fea_dict_df_list))
print(len(set(fea_dict_df_list)))

6990
6990


In [5]:
fea_ch_dict = fea_dict_df[['feature_id','description']].set_index('feature_id')['description'].to_dict()
print(len(fea_ch_dict))
fea_ch_dict

6990


{'app_7_newinstall_F010101': '近7天，网络小贷-持牌类APP新增款数',
 'app_7_newinstall_F010102': '近7天，网络小贷-非持牌类APP新增款数',
 'app_7_newinstall_F010103': '近7天，P2P网贷类APP新增款数',
 'app_7_newinstall_F010104': '近7天，持牌消金类APP新增款数',
 'app_7_newinstall_F010105': '近7天，贷款超市类APP新增款数',
 'app_7_newinstall_F010106': '近7天，抵押贷款类APP新增款数',
 'app_7_newinstall_F010107': '近7天，汽车金融车贷类APP新增款数',
 'app_7_newinstall_F010108': '近7天，征信类APP新增款数',
 'app_7_newinstall_F010109': '近7天，信用卡类APP新增款数',
 'app_7_newinstall_F010110': '近7天，高风险类APP新增款数',
 'app_7_newinstall_F010111': '近7天，其他贷款类APP新增款数',
 'app_7_newinstall_F010304': '近7天，公积金类APP新增款数',
 'app_7_newinstall_F010401': '近7天，体育彩票类APP新增款数',
 'app_7_newinstall_F010402': '近7天，福利彩票类APP新增款数',
 'app_7_newinstall_F010403': '近7天，其他彩票类APP新增款数',
 'app_7_newinstall_F010501': '近7天，证券类APP新增款数',
 'app_7_newinstall_F010502': '近7天，股票类APP新增款数',
 'app_7_newinstall_F010503': '近7天，基金类APP新增款数',
 'app_7_newinstall_F010504': '近7天，期货期权类APP新增款数',
 'app_7_newinstall_F010505': '近7天，虚拟货币类APP新增款数',
 'app_7_newinstall_F0

# 读取特征

In [6]:
df_features = pd.read_csv('feature_data_7days.csv')
df_features.shape

(82361, 1706)

# 参数配置

In [7]:
cust_id = 'tdid'

exclude_vars=['apply_time', 'target']

miss_threshold=0.95

In [8]:
df_features.set_index(cust_id, inplace=True)
df_features.head(1)

,apply_time,target,app_7_days_F010401,app_7_uninstall_F010705,app_7_days_T106050,app_7_active_T118040,app_7_active_T119040,app_7_freq_T104070,app_7_days_T113010,app_7_uninstall_T120020,...,app_7_install_T104010,app_7_freq_T116999,app_7_install_F010304,app_7_freq_T116020,app_7_days_T103999,app_7_active_T115010,app_7_newinstall_F0105,app_7_days_F0105,app_7_active_F010310,app_7_newinstall_T101050
tdid,,,,,,,,,,,,,,,,,,,,,
3019fab686c84e4634a50e64563e2cd6a,2021-02-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#删除全为空的特征
selected_features = df_features.columns.format()
if exclude_vars:
    selected_features = list(set(selected_features) - set(exclude_vars))
    
print('before is : ', df_features.shape)
df_features.dropna(subset=selected_features, how='all', inplace=True)
print('after is : ', df_features.shape)

before is :  (82361, 1705)
after is :  (82361, 1705)


In [10]:
df_features = df_features[selected_features]
df_features.head(1)

,app_7_freq_F010305,app_7_freq_T108050,app_7_install_F010301,app_7_freq_T108040,app_7_freq_15,consume_7_days_382,app_7_freq_T101010,app_7_newinstall_F010105,app_7_install_T106070,app_7_uninstall_48,...,app_7_install_T110999,app_7_days_T105030,app_7_newinstall_T106050,app_7_install_T123010,app_7_newinstall_F010305,app_7_active_F0102,app_7_freq_T111030,app_7_active_T117080,consume_7_days_350,app_7_newinstall_T119090
tdid,,,,,,,,,,,,,,,,,,,,,
3019fab686c84e4634a50e64563e2cd6a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
eda = detector.detect(df_features)
eda.to_excel(base_path+'data_EDA.xlsx')

print('删除缺失前', len(selected_features))
selected_features = eda.loc[eda['missing_q'] < miss_threshold].index.tolist()
print(len(selected_features))

删除缺失前 1703
308


In [12]:
topNvar = ['app_7_install_T102020', 'app_7_install_T103020', 'app_7_install_T114999', 'app_7_install_T103010', 'app_7_install_F010602', 'app_7_install_T118060', 'app_7_install_T114020', 'app_7_install_15', 'app_7_install_T103000', 'app_7_install_1201', 'app_7_install_T102030', 'app_7_install_T114070', 'app_7_install_T115040', 'app_7_install_T103040', 'app_7_install_T120030', 'app_7_install_T119020', 'app_7_install_T103030', 'app_7_install_F0106', 'app_7_install_T114000', 'app_7_install_T120010', 'app_7_install_T103999', 'app_7_install_F010705', 'app_7_install_T120000', 'app_7_install_T115000', 'app_7_install_T119000', 'app_7_install_T116999', 'app_7_install_F010507', 'device_public_price', 'app_7_install_T108040', 'app_7_install_T104999', 'app_7_install_T119010', 'app_7_install_T200000', 'app_7_install_T119090', 'app_7_install_T101040', 'app_7_install_T108000', 'app_7_install_F0101', 'device_osversion_diff', 'app_7_install_T117050', 'app_7_uninstall_T103010', 'app_7_uninstall_T102020', 'population_326', 'app_7_install_F0105', 'app_7_uninstall_T103000', 'device_osversion', 'app_7_install_T104000', 'app_7_install_T101010', 'app_7_install_T115020', 'app_7_install_T101999', 'app_7_install_T113000']
len(topNvar)

49

In [13]:
topNvar = ['app_7_install_T102020', 'app_7_install_T103020', 'app_7_install_T114999', 'app_7_install_T103010', 'app_7_install_F010602', 'app_7_install_T118060', 'app_7_install_T114020', 'app_7_install_15', 'app_7_install_T103000', 'app_7_install_1201', 'app_7_install_T102030', 'app_7_install_T114070', 'app_7_install_T115040', 'app_7_install_T103040', 'app_7_install_T120030', 'app_7_install_T119020', 'app_7_install_T103030', 'app_7_install_F0106', 'app_7_install_T114000', 'app_7_install_T120010', 'app_7_install_T103999', 'app_7_install_F010705', 'app_7_install_T120000', 'app_7_install_T115000', 'app_7_install_T119000', 'app_7_install_T116999', 'app_7_install_F010507', 'device_public_price', 'app_7_install_T108040', 'app_7_install_T104999', 'app_7_install_T119010', 'app_7_install_T200000', 'app_7_install_T119090', 'app_7_install_T101040', 'app_7_install_T108000', 'app_7_install_F0101', 'device_osversion_diff', 'app_7_install_T117050', 'app_7_uninstall_T103010', 'app_7_uninstall_T102020', 'population_326', 'app_7_install_F0105', 'app_7_uninstall_T103000', 'device_osversion', 'app_7_install_T104000', 'app_7_install_T101010', 'app_7_install_T115020', 'app_7_install_T101999', 'app_7_install_T113000']

df_corr_top = df_features[topNvar].corr(method='pearson')

In [14]:
df_corr_top.index = df_corr_top.index.map(lambda x: x+'&{}'.format(fea_ch_dict[x]))
df_corr_top.columns = df_corr_top.columns.map(lambda x: x+'&{}'.format(fea_ch_dict[x]))
df_corr_top.head(1)

,app_7_install_T102020&近7天，应用分发/厂商商店类APP安装款数,app_7_install_T103020&近7天，移动工具/实用工具类APP安装款数,app_7_install_T114999&近7天，汽车服务/其他汽车服务类APP安装款数,app_7_install_T103010&近7天，移动工具/系统工具类APP安装款数,app_7_install_F010602&近7天，计算器类APP安装款数,app_7_install_T118060&近7天，音乐音频/音乐播放器类APP安装款数,app_7_install_T114020&近7天，汽车服务/汽车资讯类APP安装款数,app_7_install_15&近7天，游戏类型/跑酷竞速类APP安装款数,app_7_install_T103000&近7天，移动工具类APP安装款数,app_7_install_1201&近7天，游戏类型/游戏助手类APP安装款数,...,app_7_uninstall_T102020&近7天，应用分发/厂商商店类APP卸载款数,population_326&人口属性/车辆情况,app_7_install_F0105&近7天，投资类APP安装款数,app_7_uninstall_T103000&近7天，移动工具类APP卸载款数,device_osversion&设备操作系统版本,app_7_install_T104000&近7天，旅游服务类APP安装款数,app_7_install_T101010&近7天，通讯社交/即时通讯类APP安装款数,app_7_install_T115020&近7天，健康美容/运动健身类APP安装款数,app_7_install_T101999&近7天，通讯社交/其他通讯社交类APP安装款数,app_7_install_T113000&近7天，生活服务类APP安装款数
app_7_install_T102020&近7天，应用分发/厂商商店类APP安装款数,1.0,-0.086567,0.006511,-0.387959,0.434107,0.507945,-0.06814,0.530252,-0.027963,0.53285,...,-0.12898,-0.112586,0.139688,0.454603,0.327399,0.054123,0.12088,0.290047,0.201577,0.041572


In [15]:
df_corr_top.to_excel(base_path+'df_corr_top49.xlsx')

# 根据选择好的变量生成入模df

In [16]:

filter_fea = ['device_public_price', 'app_7_install_T121000', 'app_7_install_T103010', 'device_osversion_diff', 'population_1209', 'device_9to5', 'app_7_freq_T121000', 'location_home_hospital', 'app_7_install_F010102', 'app_7_install_1201', 'device_allnight', 'app_7_install_T119020', 'app_7_install_T114999', 'app_7_install_F0101']
len(filter_fea)

14

In [17]:
components_num = 18    #暂时ok
#components_num = 6\10\12   #后续版本使用这个
#12簇 0.07
#15簇 0.07

#11簇 0.6

#14簇 0.8

#5簇 0.7

#12簇 0.9

#8簇 0.8

#19簇 1.0

#11簇  1.0    18簇


# 8   0         0.900000
#     1         0.100000
# 9   0         0.923992
#     1         0.076008
# 8       340
# 9      6052


#0.2847
# 2   0         0.918759
#     1         0.081241
# 9   0         0.908012
#     1         0.091988
# 10  0         0.895735
#     1         0.104265

# 2       677
# 9      2022
# 10      422



# 12  0         0.892791
#     1         0.107209
# 18  0         0.905788
#     1         0.094212
# 6   0         0.924056
#     1         0.075944
    
# 12     541
# 8     1325
# 6     4714



df_to_model = df_features[filter_fea]
df_to_model.shape

print('before is : ', df_to_model.shape)
df_to_model.dropna(subset=filter_fea, how='all', inplace=True)
print('after is : ', df_to_model.shape)
# before is :  (144732, 20)
# after is :  (144571, 20)

before is :  (82361, 14)
after is :  (76885, 14)


/Users/ryanzheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
df_to_model.columns

Index(['device_public_price', 'app_7_install_T121000', 'app_7_install_T103010',
       'device_osversion_diff', 'population_1209', 'device_9to5',
       'app_7_freq_T121000', 'location_home_hospital', 'app_7_install_F010102',
       'app_7_install_1201', 'device_allnight', 'app_7_install_T119020',
       'app_7_install_T114999', 'app_7_install_F0101'],
      dtype='object')

# 缺失值填充

In [19]:
fillna_value = -999
miss_le_threshould_fill_median = 0.05

# #使用中位数填充
# miss_le10_gt0 = eda.loc[(eda['missing_q'] <= miss_le_threshould_fill_median) & (eda['missing_q'] > 0) & (
#             eda['type'] != 'object')].index.tolist()  # 选择出缺失大于0小于等于10的变量
# miss_le10_gt0 = list(set(filter_fea).intersection(set(miss_le10_gt0)))
# print(len(miss_le10_gt0))
# median_fill_dict = {i: df_to_model[i].median() for i in miss_le10_gt0}
# with open(base_path + 'feature_median_mapping.json', 'w') as f:
#     json.dump(median_fill_dict, f)
# df_to_model.fillna(median_fill_dict, inplace=True)

df_to_model.fillna(fillna_value, inplace=True)
df_to_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76885 entries, 442c20d7fd35a4195232fffc84c8d15fd to 3bdd24feda24c9fab1616a0acd11e42ca
Data columns (total 14 columns):
device_public_price       76885 non-null float64
app_7_install_T121000     76885 non-null float64
app_7_install_T103010     76885 non-null float64
device_osversion_diff     76885 non-null float64
population_1209           76885 non-null float64
device_9to5               76885 non-null float64
app_7_freq_T121000        76885 non-null float64
location_home_hospital    76885 non-null float64
app_7_install_F010102     76885 non-null float64
app_7_install_1201        76885 non-null float64
device_allnight           76885 non-null float64
app_7_install_T119020     76885 non-null float64
app_7_install_T114999     76885 non-null float64
app_7_install_F0101       76885 non-null float64
dtypes: float64(14)
memory usage: 8.8+ MB


/Users/ryanzheng/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


# 采用Z-Score规范化数据，保证每个特征维度的数据均值为0，方差为1

In [20]:
stas = StandardScaler()
df_data_zscore = stas.fit_transform(df_to_model)
df_data_zscore

array([[ 0.15984604, -1.38826845, -1.02571846, ..., -0.73206378,
        -0.4462464 , -1.30516504],
       [ 0.66843208, -1.38826845, -1.02571846, ..., -0.73206378,
        -0.4462464 , -1.30516504],
       [ 0.32937472,  0.71916442,  0.98779862, ...,  1.36694369,
         2.24137294,  0.76044912],
       ...,
       [ 0.49325244,  0.70452947, -1.02571846, ..., -0.73206378,
        -0.4462464 ,  0.76251473],
       [-0.17921132, -1.38826845, -1.02571846, ..., -0.73206378,
        -0.4462464 , -1.30516504],
       [ 1.50985943,  0.70243877, -1.02571846, ..., -0.73206378,
        -0.4462464 ,  0.76044912]])

In [21]:
scaler = StandardScaler().fit(df_to_model)
df_data_zscore = scaler.transform(df_to_model)
df_data_zscore

array([[ 0.15984604, -1.38826845, -1.02571846, ..., -0.73206378,
        -0.4462464 , -1.30516504],
       [ 0.66843208, -1.38826845, -1.02571846, ..., -0.73206378,
        -0.4462464 , -1.30516504],
       [ 0.32937472,  0.71916442,  0.98779862, ...,  1.36694369,
         2.24137294,  0.76044912],
       ...,
       [ 0.49325244,  0.70452947, -1.02571846, ..., -0.73206378,
        -0.4462464 ,  0.76251473],
       [-0.17921132, -1.38826845, -1.02571846, ..., -0.73206378,
        -0.4462464 , -1.30516504],
       [ 1.50985943,  0.70243877, -1.02571846, ..., -0.73206378,
        -0.4462464 ,  0.76044912]])

In [22]:
# 构造GMM聚类并训练;n_components=6
gmm = GaussianMixture(n_components=components_num, covariance_type='full', random_state=666)
gmm.fit(df_data_zscore)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=18, n_init=1,
                precisions_init=None, random_state=666, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [23]:
# 训练数据进行预测
pred = gmm.predict(df_data_zscore)

In [24]:
#将分组结果输出到CSV文件中
df_to_model.insert(0,'分组',pred)
df_to_model.sort_values('分组').head(15)

,分组,device_public_price,app_7_install_T121000,app_7_install_T103010,device_osversion_diff,population_1209,device_9to5,app_7_freq_T121000,location_home_hospital,app_7_install_F010102,app_7_install_1201,device_allnight,app_7_install_T119020,app_7_install_T114999,app_7_install_F0101
tdid,,,,,,,,,,,,,,,
31da198a40fb42edb6b0e6ddd42412a9d,0,-999.0,-999.0,-999.0,-999.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3e403aebcb89decb2f9fc3fc3c00e73e6,0,-999.0,-999.0,-999.0,-999.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3174cb593cbf7a47d346c41ecf842ed46,0,-999.0,-999.0,-999.0,-999.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
30b1a638cf038d076babefaa85a7134e2,0,-999.0,-999.0,-999.0,-999.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4174ccfcb588753cedcf9579553e6e35c,0,-999.0,-999.0,-999.0,-999.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3450cb8cd897943ff55e586908ab32b0c,0,-999.0,-999.0,-999.0,-999.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
31e91e9885ca3fa2308227ba0fc944e1e,0,-999.0,-999.0,-999.0,-999.0,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
36a7dc790493a568e60da0ca8d194a4df,0,-999.0,-999.0,-999.0,-999.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0
35ee13c6bd263d5150d43d28e3db9c0d6,0,-999.0,-999.0,-999.0,-999.0,8.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


# 根据轮廓系数计算模型得分

In [26]:
from sklearn.metrics import silhouette_score
score=silhouette_score(df_data_zscore,pred,metric='euclidean')
score
0.258944526193597

0.258944526193597

In [27]:
df_to_model.shape

(76885, 15)

In [28]:
TD41p1mon2zjtswjd7day_id = pd.read_csv('TD41p1mon2zjtswjd7day_id.csv')
print(TD41p1mon2zjtswjd7day_id.shape)
TD41p1mon2zjtswjd7day_id.set_index(cust_id, inplace=True)
TD41p1mon2zjtswjd7day_id.head(2)

(82361, 3)


,target,apply_time
tdid,,
3019fab686c84e4634a50e64563e2cd6a,1,2021-02-01
442c20d7fd35a4195232fffc84c8d15fd,0,2021-02-01


In [29]:
TD41p1mon2zjtswjd7day_id_with_result = df_to_model.merge(TD41p1mon2zjtswjd7day_id, how='left', left_index=True, right_index=True)
TD41p1mon2zjtswjd7day_id_with_result.head(2)

,分组,device_public_price,app_7_install_T121000,app_7_install_T103010,device_osversion_diff,population_1209,device_9to5,app_7_freq_T121000,location_home_hospital,app_7_install_F010102,app_7_install_1201,device_allnight,app_7_install_T119020,app_7_install_T114999,app_7_install_F0101,target,apply_time
tdid,,,,,,,,,,,,,,,,,
442c20d7fd35a4195232fffc84c8d15fd,7,2599.0,-999.0,-999.0,0.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,2021-02-01
3cc2a25de6de820f48426f3d1b25e88b7,17,3499.0,-999.0,-999.0,3.0,2.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,2021-02-01


In [30]:
TD41p1mon2zjtswjd7day_id_with_result.shape

(76885, 17)

In [31]:
TD41p1mon2zjtswjd7day_id_with_result.reset_index(inplace=True)
TD41p1mon2zjtswjd7day_id_with_result.head(2)

,tdid,分组,device_public_price,app_7_install_T121000,app_7_install_T103010,device_osversion_diff,population_1209,device_9to5,app_7_freq_T121000,location_home_hospital,app_7_install_F010102,app_7_install_1201,device_allnight,app_7_install_T119020,app_7_install_T114999,app_7_install_F0101,target,apply_time
0,442c20d7fd35a4195232fffc84c8d15fd,7,2599.0,-999.0,-999.0,0.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,2021-02-01
1,3cc2a25de6de820f48426f3d1b25e88b7,17,3499.0,-999.0,-999.0,3.0,2.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0,2021-02-01


In [32]:
TD41p1mon2zjtswjd7day_id_with_result.groupby(['分组','target'])['tdid'].count()

分组  target
0   0          2758
    1           142
1   0          3181
    1           133
2   0           622
    1            55
3   0          1740
    1            48
4   0         12057
    1           627
5   0          6153
    1           249
6   0          3263
    1           146
7   0          6931
    1           336
8   0          2074
    1            14
9   0          1836
    1           186
10  0           378
    1            44
11  0          1881
    1            13
12  0         10779
    1           503
13  0          1633
    1            76
14  0          3832
    1           177
15  0          7042
    1           416
16  0          3565
    1            35
17  0          3778
    1           182
Name: tdid, dtype: int64

In [33]:
TD41p1mon2zjtswjd7day_id_with_result.groupby(['分组'])['tdid'].count()

分组
0      2900
1      3314
2       677
3      1788
4     12684
5      6402
6      3409
7      7267
8      2088
9      2022
10      422
11     1894
12    11282
13     1709
14     4009
15     7458
16     3600
17     3960
Name: tdid, dtype: int64

In [34]:
TD41p1mon2zjtswjd7day_id_with_result['target'].value_counts()/len(TD41p1mon2zjtswjd7day_id_with_result)

0    0.956012
1    0.043988
Name: target, dtype: float64

In [35]:
TD41p1mon2zjtswjd7day_id_with_result.groupby(['分组','target'])['tdid'].count()/TD41p1mon2zjtswjd7day_id_with_result.groupby(['分组'])['tdid'].count()

分组  target
0   0         0.951034
    1         0.048966
1   0         0.959867
    1         0.040133
2   0         0.918759
    1         0.081241
3   0         0.973154
    1         0.026846
4   0         0.950568
    1         0.049432
5   0         0.961106
    1         0.038894
6   0         0.957172
    1         0.042828
7   0         0.953764
    1         0.046236
8   0         0.993295
    1         0.006705
9   0         0.908012
    1         0.091988
10  0         0.895735
    1         0.104265
11  0         0.993136
    1         0.006864
12  0         0.955416
    1         0.044584
13  0         0.955530
    1         0.044470
14  0         0.955849
    1         0.044151
15  0         0.944221
    1         0.055779
16  0         0.990278
    1         0.009722
17  0         0.954040
    1         0.045960
Name: tdid, dtype: float64

# 模型保存

In [36]:
import pickle

In [37]:
with open(base_path + 'gmm.pickle', 'wb') as f:
    pickle.dump(gmm, f)

In [38]:
#scaler
with open(base_path + 'scaler.pickle', 'wb') as f:
    pickle.dump(scaler, f)